In [1]:
import pennylane as qml
from pennylane import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# 1) Load data
iris = datasets.load_iris()
X = iris.data  # (150,4)
y = iris.target  # (150,)

# 2) Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 3) Standardize features (helps quantum encoding)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# 4) Define quantum devices and helpers (keep options simple)

n_qubits = 4  # one qubit per feature
dev_basic = qml.device("default.qubit", wires=n_qubits)

# Encoding 1: Simple angle encoding with a minimal trainable layer
def feature_map_simple(x):
    for i in range(n_qubits):
        qml.RY(x[i], wires=i)

def circuit_simple(params, x=None):
    feature_map_simple(x)
    # small trainable layer
    for i in range(n_qubits):
        qml.RY(params[i], wires=i)
    # light entanglement
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i+1])
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

@qml.qnode(dev_basic)
def embedding_simple(x, params):
    return circuit_simple(params, x)

# Encoding 2: Feature map with Hadamards and RX for variety (no heavy compute)
def feature_map_hadamard(x):
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
        qml.RX(x[i], wires=i)

@qml.qnode(dev_basic)
def embedding_hadamard(x, _=None):
    feature_map_hadamard(x)
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# Encoding 3: Simple parameterized circuit (more trainable depth)
def feature_map_param(x):
    for i in range(n_qubits):
        qml.RY(x[i], wires=i)

@qml.qnode(dev_basic)
def circuit_param(params, x=None):
    feature_map_param(x)
    # one more layer of rotations with shared params
    for i in range(n_qubits):
        qml.RY(params[i], wires=i)
    # simple entangling ring
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i+1])
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

@qml.qnode(dev_basic)
def embedding_param(x, params):
    return circuit_param(params, x)

# 5) Prepare a tiny function to create embeddings from a dataset
def make_embeddings(X, method, param=None):
    emb = []
    if method == "simple":
        for xi in X:
            emb.append(embedding_simple(xi, param))
        return np.array(emb)
    elif method == "hadamard":
        for xi in X:
            emb.append(embedding_hadamard(xi))
        return np.array(emb)
    elif method == "param":
        for xi in X:
            emb.append(embedding_param(xi, param))
        return np.array(emb)
    else:
        raise ValueError("Unknown embedding method")

# 6) Initialize parameters for each method
np.random.seed(0)
params_simple = np.random.uniform(-0.5, 0.5, size=(n_qubits,))
params_param = np.random.uniform(-0.5, 0.5, size=(n_qubits,))

# 7) Build embeddings
X_train_emb_simple = make_embeddings(X_train_scaled, "simple", params_simple)
X_test_emb_simple  = make_embeddings(X_test_scaled, "simple", params_simple)

X_train_emb_hadamard = make_embeddings(X_train_scaled, "hadamard")
X_test_emb_hadamard  = make_embeddings(X_test_scaled, "hadamard")

X_train_emb_param = make_embeddings(X_train_scaled, "param", params_param)
X_test_emb_param  = make_embeddings(X_test_scaled, "param", params_param)

# 8) Optional: PCA on embeddings (2 components for visualization/compactness)
pca_components = 2

def apply_pca(train_emb, test_emb):
    pca = PCA(n_components=pca_components)
    train_pca = pca.fit_transform(train_emb)
    test_pca  = pca.transform(test_emb)
    return train_pca, test_pca, pca

# Apply PCA to each embedding family
t1, te1, pca1 = apply_pca(X_train_emb_simple, X_test_emb_simple)
t2, te2, pca2 = apply_pca(X_train_emb_hadamard, X_test_emb_hadamard)
t3, te3, pca3 = apply_pca(X_train_emb_param, X_test_emb_param)

# 9) Train classifiers on embeddings (and compare to baseline on original features)
def train_eval(train_X, test_X, y_train, y_test):
    clf = LogisticRegression(max_iter=200, multi_class="auto")
    clf.fit(train_X, y_train)
    pred = clf.predict(test_X)
    acc  = accuracy_score(y_test, pred)
    return acc

# Baseline with original scaled features
acc_orig = train_eval(X_train_scaled, X_test_scaled, y_train, y_test)

# Embedding baselines (PCA-reduced embeddings)
acc_simple_pca = train_eval(t1, te1, y_train, y_test)
acc_hadamard_pca = train_eval(t2, te2, y_train, y_test)
acc_param_pca = train_eval(t3, te3, y_train, y_test)

# 10) Print results
print("Baseline (original features) accuracy: {:.2f}%".format(acc_orig * 100))
print("Simple embedding + PCA accuracy: {:.2f}%".format(acc_simple_pca * 100))
print("Hadamard embedding + PCA accuracy: {:.2f}%".format(acc_hadamard_pca * 100))
print("Parametrized embedding + PCA accuracy: {:.2f}%".format(acc_param_pca * 100))

# 11) Plot results
methods = ["Original features", "Simple PCA Embedding", "Hadamard PCA Embedding", "Param PCA Embedding"]
accs = [acc_orig, acc_simple_pca, acc_hadamard_pca, acc_param_pca]

plt.figure(figsize=(8,5))
bars = plt.bar(methods, [a*100 for a in accs], color=["C0","C1","C2","C3"])
plt.ylabel("Accuracy %")
plt.title("Classification accuracy across embedding methods with PCA (Iris)")
for b, a in zip(bars, accs):
    plt.text(b.get_x() + b.get_width()/2, b.get_height() + 1, f"{a*100:.2f}%", ha="center", va="bottom")
plt.ylim(0, 100)
plt.grid(axis="y", alpha=0.2)
plt.tight_layout()
plt.show()


TypeError: iteration over a 0-d array